In [1]:
import sys
print(sys.executable)

/home/ubuntu/miniconda3/envs/llm_tools1/bin/python


In [1]:
from llama_cpp import Llama
import json
import pandas as pd

In [2]:
path_to_model = '/home/ubuntu/Final_project/openhermes-2.5-mistral-7b.Q2_K.gguf'
#path_to_model = '/mnt/efs/home/ubuntu/Final_project/medgemma-27b-it-Q5_K_M.gguf'
llm = Llama(model_path = path_to_model, 
            n_ctx = 9999, 
            max_new_tokens = 2048, 
            temperature = 0.1, 
            cache = False, 
            verbose = False,
            n_gpu_layers = -1,
            n_threads = 6,
            chat_format = 'chatml')


llama_context: n_ctx_per_seq (9999) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [3]:
# label_format = {
#     "type": "json_object",
#     "schema": {
#         "type": "object",
#         "properties": {
#             "imatinib_mentioned": {
#                 "type": "boolean",
#                 "description": "True if the drug imatinib is mentioned in the note"
#             },
#             "cml_diagnosed": {
#                 "type": "boolean",
#                 "description": "True if chronic myeloid leukemia is diagnosed"
#             },
#             "cml_in_regression": {
#                 "type": "boolean",
#                 "description": "True if chronic myeloid leukemia is mentioned as being in regression"
#             }
#         },
#         "required": ["imatinib_mentioned", "cml_diagnosed", "cml_in_regression"]
#     }
# }

In [4]:
# system_prompt = """
# You are a strict JSON generator. Only output JSON that matches the schema below.
# Do not include any extra text, commentary, or explanations.

# Schema:
# - imatinib_mentioned: true if the drug imatinib is mentioned in the note, otherwise false
# - cml_diagnosed: true if chronic myeloid leukemia is diagnosed, otherwise false
# - cml_in_regression: true if chronic myeloid leukemia is mentioned as being in regression, otherwise false

# Rules:
# 1. Only mark a field as true if the note clearly indicates it.
# 2. If the note does not explicitly mention a field, mark it false.
# 3. The output must always be valid JSON with all three fields present.
# """

In [5]:
label_format = {
    "type": "json_object",
    "schema": {
        "type": "object",
        "properties": {
            "imatinib_mentioned": {
                "type": "boolean",
                "description": "True if the drug imatinib is mentioned in the note"
            },
            "related_drugs_mentioned": {
                "type": "boolean",
                "description": "True if drugs related to imatinib (e.g., dasatinib, nilotinib, bosutinib) are mentioned"
            },
            "cml_diagnosed": {
                "type": "boolean",
                "description": "True if chronic myeloid leukemia is diagnosed"
            },
            "cml_in_regression": {
                "type": "boolean",
                "description": "True if chronic myeloid leukemia is mentioned as being in regression"
            }
        },
        "required": ["imatinib_mentioned", "related_drugs_mentioned", "cml_diagnosed", "cml_in_regression"]
    }
}


In [6]:
system_prompt = """
You are a strict JSON generator. Only output JSON that matches the schema below.
Do not include any extra text, commentary, or explanations.

Schema:
- imatinib_mentioned: true if the drug imatinib is mentioned in the note, otherwise false
- related_drugs_mentioned: true if drugs related to imatinib (e.g., dasatinib, nilotinib, bosutinib) are mentioned, otherwise false
- cml_diagnosed: true if chronic myeloid leukemia is diagnosed, otherwise false
- cml_in_regression: true if chronic myeloid leukemia is mentioned as being in regression, otherwise false

Rules:
1. Only mark a field as true if the note clearly indicates it.
2. If the note does not explicitly mention a field, mark it false.
3. The output must always be valid JSON with all four fields present.
"""


In [7]:
# sample_notes = clinical_notes_all_path = "latest_notes.parquet"
# clinical_notes_all = pd.read_parquet(clinical_notes_all_path, engine='pyarrow')
# clinical_notes_all.columns
# clinical_notes_all['note_text']
# first_300 = clinical_notes_all['note_text']
# first_300


0       |                           ***** Data extract...
1       |                           ***** Data extract...
2       |                           ***** Data extract...
3         Subjective:     The patient returns for foll...
4       REPORT:          VISIT SUMMARY - HEMATOLOGY/ON...
                              ...                        
1757    Received admissions/referral packet from *****...
1758                                                   \n
1759    Today I ***** ***** *****, ***** ***** y.o. fe...
1760    Formatting of this note is different from the ...
1761    Formatting of this note is different from the ...
Name: note_text, Length: 1762, dtype: object

In [8]:
df = pd.read_parquet('latest_notes.parquet')
#df = df.iloc[:40]



In [26]:
# import re

# sample_notes = df['note_text'].values
# labelled_notes = []

# for i, note in enumerate(sample_notes, 1):
#     if len(note) > 11000:
#         note = note[:11000]

#     structured_output = llm.create_chat_completion(
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": note}
#         ],
#         response_format=label_format,
#         temperature=0,
#     )

#     raw_content = structured_output['choices'][0]['message']['content']

#     # extract first {...} block
#     match = re.search(r"\{.*\}", raw_content, re.DOTALL)
#     if match:
#         label = json.loads(match.group())
#     else:
#         # fallback if JSON not found
#         label = {
#             "imatinib_mentioned": False,
#             "cml_diagnosed": False,
#             "cml_in_regression": False
#         }

#     labelled_notes.append({
#         "original_note": note,
#         "imatinib_mentioned": label["imatinib_mentioned"],
#         "cml_diagnosed": label["cml_diagnosed"],
#         "cml_in_regression": label["cml_in_regression"],
#     })

#     if i % 10 == 0:
#         print(f"Processed {i} / {len(sample_notes)} notes")


Processed 10 / 40 notes
Processed 20 / 40 notes
Processed 30 / 40 notes
Processed 40 / 40 notes


In [27]:
# with open("open_hermes_test.json", "w") as f:
#     json.dump(labelled_notes, f, indent=2)

In [11]:
#import json
import re

sample_notes = df['note_text'].values
labelled_notes = []

for i, note in enumerate(sample_notes, 1):
    # truncate long notes
    if len(note) > 11000:
        note = note[:11000]

    structured_output = llm.create_chat_completion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": note}
        ],
        response_format=label_format,
        temperature=0,
    )

    raw_content = structured_output['choices'][0]['message']['content']

    # extract first JSON object to avoid JSONDecodeError
    match = re.search(r"\{.*\}", raw_content, re.DOTALL)
    if match:
        label = json.loads(match.group())
    else:
        # fallback if JSON not found
        label = {
            "imatinib_mentioned": False,
            "related_drugs_mentioned": False,
            "cml_diagnosed": False,
            "cml_in_regression": False
        }

    labelled_notes.append({
        "original_note": note,
        "imatinib_mentioned": label["imatinib_mentioned"],
        "related_drugs_mentioned": label["related_drugs_mentioned"],
        "cml_diagnosed": label["cml_diagnosed"],
        "cml_in_regression": label["cml_in_regression"],
    })

    # status update every 10 notes
    if i % 10 == 0:
        print(f"Processed {i} / {len(sample_notes)} notes")


Processed 10 / 1762 notes
Processed 20 / 1762 notes
Processed 30 / 1762 notes
Processed 40 / 1762 notes
Processed 50 / 1762 notes
Processed 60 / 1762 notes
Processed 70 / 1762 notes
Processed 80 / 1762 notes
Processed 90 / 1762 notes
Processed 100 / 1762 notes
Processed 110 / 1762 notes
Processed 120 / 1762 notes
Processed 130 / 1762 notes
Processed 140 / 1762 notes
Processed 150 / 1762 notes
Processed 160 / 1762 notes
Processed 170 / 1762 notes
Processed 180 / 1762 notes
Processed 190 / 1762 notes
Processed 200 / 1762 notes
Processed 210 / 1762 notes
Processed 220 / 1762 notes
Processed 230 / 1762 notes
Processed 240 / 1762 notes
Processed 250 / 1762 notes
Processed 260 / 1762 notes
Processed 270 / 1762 notes
Processed 280 / 1762 notes
Processed 290 / 1762 notes
Processed 300 / 1762 notes
Processed 310 / 1762 notes
Processed 320 / 1762 notes
Processed 330 / 1762 notes
Processed 340 / 1762 notes
Processed 350 / 1762 notes
Processed 360 / 1762 notes
Processed 370 / 1762 notes
Processed 

In [12]:
with open("open_hermes_test.json", "w") as f:
    json.dump(labelled_notes, f, indent=2)